In [1]:
import torch.nn as nn
import torchvision
from torchvision.models import resnet50, resnet18, squeezenet1_1
from torchvision.models.resnet import ResNet50_Weights, ResNet18_Weights
from torchvision.models import efficientnet_v2_l
from torchvision.models.efficientnet import EfficientNet_V2_L_Weights
from torchvision.models import vgg16, vgg19
from torchvision.models.vgg import VGG16_Weights, VGG19_Weights
from torchvision.models.squeezenet import SqueezeNet1_1_Weights


import glob
import torch
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import tools

2023-05-28 12:56:20.624668: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-28 12:56:21.579946: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
class Vgg_face_dag(nn.Module):


    def __init__(self):
        super(Vgg_face_dag, self).__init__()
        self.meta = {'mean': [129.186279296875, 104.76238250732422, 93.59396362304688],
                     'std': [1, 1, 1],
                     'imageSize': [224, 224, 3]}
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu1_1 = nn.ReLU(inplace=True)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu1_2 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu2_1 = nn.ReLU(inplace=True)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu2_2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu3_1 = nn.ReLU(inplace=True)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu3_2 = nn.ReLU(inplace=True)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu3_3 = nn.ReLU(inplace=True)
        self.pool3 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu4_1 = nn.ReLU(inplace=True)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu4_2 = nn.ReLU(inplace=True)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu4_3 = nn.ReLU(inplace=True)
        self.pool4 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu5_1 = nn.ReLU(inplace=True)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu5_2 = nn.ReLU(inplace=True)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
        self.relu5_3 = nn.ReLU(inplace=True)
        self.pool5 = nn.MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
        self.fc6 = nn.Linear(in_features=25088, out_features=4096, bias=True)
        self.relu6 = nn.ReLU(inplace=True)
        self.dropout6 = nn.Dropout(p=0.5)
        self.fc7 = nn.Linear(in_features=4096, out_features=4096, bias=True)
        self.relu7 = nn.ReLU(inplace=True)
        self.dropout7 = nn.Dropout(p=0.5)
        self.fc8 = nn.Linear(in_features=4096, out_features=2622, bias=True)

    def forward(self, x0):
        x1 = self.conv1_1(x0)
        x2 = self.relu1_1(x1)
        x3 = self.conv1_2(x2)
        x4 = self.relu1_2(x3)
        x5 = self.pool1(x4)
        x6 = self.conv2_1(x5)
        x7 = self.relu2_1(x6)
        x8 = self.conv2_2(x7)
        x9 = self.relu2_2(x8)
        x10 = self.pool2(x9)
        x11 = self.conv3_1(x10)
        x12 = self.relu3_1(x11)
        x13 = self.conv3_2(x12)
        x14 = self.relu3_2(x13)
        x15 = self.conv3_3(x14)
        x16 = self.relu3_3(x15)
        x17 = self.pool3(x16)
        x18 = self.conv4_1(x17)
        x19 = self.relu4_1(x18)
        x20 = self.conv4_2(x19)
        x21 = self.relu4_2(x20)
        x22 = self.conv4_3(x21)
        x23 = self.relu4_3(x22)
        x24 = self.pool4(x23)
        x25 = self.conv5_1(x24)
        x26 = self.relu5_1(x25)
        x27 = self.conv5_2(x26)
        x28 = self.relu5_2(x27)
        x29 = self.conv5_3(x28)
        x30 = self.relu5_3(x29)
        x31_preflatten = self.pool5(x30)
        x31 = x31_preflatten.view(x31_preflatten.size(0), -1)
        x32 = self.fc6(x31)
        x33 = self.relu6(x32)
        x34 = self.dropout6(x33)
        x35 = self.fc7(x34)
        x36 = self.relu7(x35)
        x37 = self.dropout7(x36)
        x38 = self.fc8(x37)
        return x38

def vgg_face_dag(weights_path='/home/disi/Project-IML/vgg_face_dag.pth'):

    model = Vgg_face_dag()
    if weights_path:
        state_dict = torch.load(weights_path)
        model.load_state_dict(state_dict)
    return model

In [2]:
class LaBocciaModel(nn.Module):
    def __init__(self):
        super(LaBocciaModel, self).__init__()
        #self.pretrained = squeezenet1_1(weights = SqueezeNet1_1_Weights.DEFAULT)
        #self.pretrained = resnet50(weights = ResNet50_Weights.DEFAULT).eval()
        #self.pretrained = vgg16(weights = VGG16_Weights.DEFAULT).eval()
        #self.pretrained = vgg19(weights = VGG19_Weights.DEFAULT).eval()
        #self.pretrained = resnet18(weights = ResNet18_Weights.DEFAULT).eval()
        self.pretrained = efficientnet_v2_l(weights = EfficientNet_V2_L_Weights.DEFAULT).eval()
        self.pretrained.fc = nn.Identity()

    def forward(self, x):
        x = self.pretrained(x)
        return x

In [3]:
from datetime import datetime
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import datasets
from train_functions import *

device = "cuda:0" if torch.cuda.is_available() else "cpu"

cudnn.benchmark = True


train_path = "/run/user/1000/train_dataset/"
test_path = "/run/user/1000/test_dataset/"


t = torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Resize((224, 224), antialias=True),
            torchvision.transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
        ])

batch_size = 512

trainset = datasets.ImageFolder(root=train_path, transform=t)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)
testset = datasets.ImageFolder(root=test_path, transform=t)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4)
classes = len(trainset.classes)
net = Vgg_face_dag()
scheduler = adjust_lr(50, 0.1)
lr = 0.005
epochs = 10
inner_param = 0.5
sigma = 0.25
hlen = 48
eta = 0.01
save = "/home/disi/Project-IML/"
train(epochs, net, device, trainloader, testloader, classes, scheduler, lr, inner_param, sigma, hlen, eta, save)


==> Preparing training data..
==> Epoch: 1
(512, 2622)
torch.Size([512])
torch.Size([512, 3, 224, 224])
cpu


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [3]:

input_dir = "/home/disi/Project-IML/query/"
output_dir = "/home/disi/Project-IML/aligned_query/"

tools.face_alignment(input_dir, output_dir)

input_dir = "/home/disi/Project-IML/gallery/"
output_dir = "/home/disi/Project-IML/aligned_gallery/"

tools.face_alignment(input_dir, output_dir)

2023-05-16 15:37:38.488311: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-16 15:37:45.209611: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
class TrainDataset(torch.utils.data.Dataset):

  def __init__(self, root: str):

    self.root = root

    #self.split_ids = io.loadmat(os.path.join(root, "setid.mat"))
    #self.labels = io.loadmat(os.path.join(root, "imagelabels.mat"))["labels"][0]
    self.split_ids = {}
    self.image_paths = sorted(glob.glob(os.path.join(self.root, "jpg", "*.jpg")))

  def __len__(self):
    return len(self.image_paths)

  def __getitem__(self, idx: int):

    label = self.labels[idx]
    img = Image.open(self.image_paths[idx]).convert("RGB")

    return img, label


In [4]:
# PER LA BOCCIA

import pickle

data_dir = "/home/disi/Project-IML/aligned_gallery/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gallery_paths = glob.glob(f"{data_dir}*", recursive=True)

model = LaBocciaModel()
model.to(device)

embeddings = tools.embeddings_calc(gallery_paths, model, device, data_dir)


# save embeddings to disk
with open("/home/disi/Project-IML/embeddings.pkl", "wb") as f:
    pickle.dump(embeddings, f)

data_dir = "/home/disi/Project-IML/aligned_query/"

query_paths = glob.glob(f"{data_dir}*", recursive=True)

queries = tools.embeddings_calc(query_paths, model, device, data_dir)


In [4]:
## PER VGG FACE DAG

data_dir = "/home/disi/Project-IML/aligned_gallery/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gallery_paths = glob.glob(f"{data_dir}*", recursive=True)

model = vgg_face_dag()
model.eval()
model.to(device)

embeddings = tools.embeddings_calc(gallery_paths, model, device, data_dir)


data_dir = "/home/disi/Project-IML/aligned_query/"

query_paths = glob.glob(f"{data_dir}*", recursive=True)

queries = tools.embeddings_calc(query_paths, model, device, data_dir)


In [5]:
def euclidian_distance(x, y):
    return np.sqrt(np.sum((x - y)**2))

def extract_id(string):
    splitter = string.split('/')
    splitter = splitter[-1].split('_')
    splitter = splitter[0]

    return splitter

def number_equal(sort):
    query_num = 0

    for i in sort:
        i_id = extract_id(i[0])
        if i_id == query_id:
            query_num += 1

    return query_num

query_num_list_10 = []
query_num_list_5 = []
query_num_list_1 = []

for query in queries:
    query_embedding = queries[query]
    distances = {}
    counter = {}
    for gallery in embeddings:
        gallery_embedding = embeddings[gallery]
        distances[gallery] = euclidian_distance(query_embedding, gallery_embedding)

        gallery_id = extract_id(gallery)
        if gallery_id in counter:
            counter[gallery_id] += 1
        else:
            counter[gallery_id] = 1

    sorted_distances = sorted(distances.items(), key=lambda x: x[1])
    #print(f"Query: {query}")
    #print(sorted_distances[:5])
    sorted_distances_10 = sorted_distances[:10]
    sorted_distances_5 = sorted_distances[:5]
    sorted_distances_1 = sorted_distances[:1]

    query_id = extract_id(query)

    query_num_10 = number_equal(sorted_distances_10)
    query_num_5 = number_equal(sorted_distances_5)
    query_num_1 = number_equal(sorted_distances_1)

    query_num_list_10.append(query_num_10)
    query_num_list_5.append(query_num_5)
    query_num_list_1.append(query_num_1)
    #print(f'Gallery: {counter[query_id]}, found: {query_num}')

print(f'Raw accuracy top 10 euclidian distance: {np.array(query_num_list_10).mean()/np.array(list(counter.values())).mean()}')
print(f'Raw accuracy top 5 euclidian distance: {np.array(query_num_list_5).mean()/min(np.array(list(counter.values())).mean(),5)}')
print(f'Raw accuracy top 1 euclidian distance: {np.array(query_num_list_1).mean()/min(np.array(list(counter.values())).mean(),1)}')

# now again but with another distance metric

from sklearn.metrics.pairwise import cosine_similarity


query_num_list_10 = []
query_num_5 = []
query_num_1 = []
for query in queries:
    query_embedding = queries[query]
    distances = {}
    for gallery in embeddings:
        gallery_embedding = embeddings[gallery]
        distances[gallery] = cosine_similarity(query_embedding, gallery_embedding)[0][0]
    sorted_distances = sorted(distances.items(), key=lambda x: x[1], reverse=True)
    #print(f"Query: {query}")
    #print(sorted_distances[:5])

    sorted_distances_10 = sorted_distances[:10]
    sorted_distances_5 = sorted_distances[:5]
    sorted_distances_1 = sorted_distances[:1]

    query_id = extract_id(query)

    query_num_10 = number_equal(sorted_distances_10)
    query_num_5 = number_equal(sorted_distances_5)
    query_num_1 = number_equal(sorted_distances_1)

    query_num_list_10.append(query_num_10)
    query_num_list_5.append(query_num_5)
    query_num_list_1.append(query_num_1)
    #print(f'Gallery: {counter[query_id]}, found: {query_num}')

print(f'Raw accuracy top 10 cosine similarity: {np.array(query_num_list_10).mean()/np.array(list(counter.values())).mean()}')
print(f'Raw accuracy top 5 cosine similarity: {np.array(query_num_list_5).mean()/min(np.array(list(counter.values())).mean(),5)}')
print(f'Raw accuracy top 1 cosine similarity: {np.array(query_num_list_1).mean()/min(np.array(list(counter.values())).mean(),1)}')


Raw accuracy top 10 euclidian distance: 0.1836694418335502
Raw accuracy top 5 euclidian distance: 0.4298850574712644
Raw accuracy top 1 euclidian distance: 0.5402298850574713
Raw accuracy top 10 cosine similarity: 0.21036188666749175
Raw accuracy top 5 cosine similarity: 0.45632183908045976
Raw accuracy top 1 cosine similarity: 0.5977011494252874


VGGFace pretrained:
Raw accuracy top 10 euclidian distance: 0.7358490566037736
Raw accuracy top 5 euclidian distance: 0.7
Raw accuracy top 1 euclidian distance: 1.0
Raw accuracy top 10 cosine similarity: 0.7735849056603773
Raw accuracy top 5 cosine similarity: 0.74
Raw accuracy top 1 cosine similarity: 1.0


EfficientNet:

Raw accuracy top 10 euclidian distance: 0.5943396226415094
Raw accuracy top 5 euclidian distance: 0.51
Raw accuracy top 1 euclidian distance: 1.0
Raw accuracy top 10 cosine similarity: 0.6603773584905661
Raw accuracy top 5 cosine similarity: 0.52
Raw accuracy top 1 cosine similarity: 1.0

VGG19

Raw accuracy top 10 euclidian distance: 0.37735849056603776
Raw accuracy top 5 euclidian distance: 0.26
Raw accuracy top 1 euclidian distance: 1.0
Raw accuracy top 10 cosine similarity: 0.44339622641509435
Raw accuracy top 5 cosine similarity: 0.27999999999999997
Raw accuracy top 1 cosine similarity: 1.0

Resnet50

Raw accuracy top 10 euclidian distance: 0.4716981132075472
Raw accuracy top 5 euclidian distance: 0.35
Raw accuracy top 1 euclidian distance: 1.0
Raw accuracy top 10 cosine similarity: 0.4905660377358491
Raw accuracy top 5 cosine similarity: 0.37
Raw accuracy top 1 cosine similarity: 1.0

VGG16

Raw accuracy top 10 euclidian distance: 0.38679245283018865
Raw accuracy top 5 euclidian distance: 0.29
Raw accuracy top 1 euclidian distance: 1.0
Raw accuracy top 10 cosine similarity: 0.37735849056603776
Raw accuracy top 5 cosine similarity: 0.29500000000000004
Raw accuracy top 1 cosine similarity: 1.0

In [ ]:
# iterate over the queries
# set the number of retrieved images to display
num_retrievals = 5

# iterate over the queries
for query in queries:
    # get the query embedding and calculate distances
    query_embedding = queries[query]
    distances = {}
    for gallery in embeddings:
        gallery_embedding = embeddings[gallery]
        distances[gallery] = cosine_similarity(query_embedding, gallery_embedding)[0][0]

    # sort the distances in descending order
    sorted_distances = sorted(distances.items(), key=lambda x: x[1], reverse=True)

    # get the top retrieved image paths and distances
    top_paths = [x[0] for x in sorted_distances[:num_retrievals]]
    top_similarities = [x[1]*100 for x in sorted_distances[:num_retrievals]]

    # load the query image and the top retrieved images
    images = [Image.open(query)] + [Image.open(path) for path in top_paths]

    # create a figure with subplots for each image
    fig, axes = plt.subplots(1, num_retrievals+1, figsize=(15, 5))

    # display the query image
    axes[0].imshow(np.array(images[0]))
    axes[0].set_title("Query")

    # display the retrieved images and their distances
    for i in range(num_retrievals):
        axes[i+1].imshow(np.array(images[i+1]))
        axes[i+1].set_title(f"Similarity: {top_similarities[i]:.2f}%")
    plt.show()


# set the number of retrieved images to display

# iterate over the queries
for query in queries:
    # get the query embedding and calculate distances
    query_embedding = queries[query]
    distances = {}
    for gallery in embeddings:
        gallery_embedding = embeddings[gallery]
        distances[gallery] = euclidian_distance(query_embedding, gallery_embedding)

    # sort the distances in ascending order
    sorted_distances = sorted(distances.items(), key=lambda x: x[1])

    # get the top retrieved image paths and distances
    top_paths = [x[0] for x in sorted_distances[:num_retrievals]]
    top_distances = [x[1] for x in sorted_distances[:num_retrievals]]

    # load the query image and the top retrieved images
    images = [Image.open(query)] + [Image.open(path) for path in top_paths]

    # create a figure with subplots for each image
    fig, axes = plt.subplots(1, num_retrievals+1, figsize=(15, 5))

    # display the query image
    axes[0].imshow(np.array(images[0]))
    axes[0].set_title("Query")

    # display the retrieved images and their similarities
    for i in range(num_retrievals):
        axes[i+1].imshow(np.array(images[i+1]))
        axes[i+1].set_title(f"EuclidianD: {top_distances[i]:.2f}%")
    plt.show()

The cosine_similarity function returns a matrix of pairwise cosine similarities between the query embedding and the gallery embeddings. Since we are comparing one query image with multiple gallery images, we need to take the first element of the first dimension to get a single similarity score. Also, note that we pass reverse=True to sorted function to sort the distances in descending order since cosine similarity values range between -1 and 1, with higher values indicating more similarity.